# Pro Trader pure implementation: no broker algo, requires manual trading

In [ ]:
from environment import BuyKnowledge, SellKnowledge, run_agent
from preprocessing import donchian_signals, test_wrapper, get_data
from datetime import datetime, timedelta

import yfinance as yf
from stable_baselines3 import PPO
import pandas as pd
import random

In [ ]:
today = datetime.now().date()
start = str(today - timedelta(days = 571))
end = str(today)

In [ ]:
buy_symbols = []
sideways_20 = {}
current_price = {}

symbol_list = [] 
for i in [5,4,6]:
    a = pd.read_excel(f'SP{i}00.xlsx')
    symbol_list += list(a.Symbol)

symbol_list = list(set(symbol_list))

In [ ]:
for symbol in symbol_list: 
    df = donchian_signals(symbol)
    if len(df)>0:
        current_price[symbol] = df.iloc[-1].close
        if df.iloc[-1].buy_signal and not df.iloc[-2].buy_signal:
            buy_symbols.append(symbol)

In [ ]:
sell_symbols = []
sell_returns = []
sell_data = pd.DataFrame()
for symbol in balance:
    last_buy_price = balance[symbol][0]

    stop_loss_data = get_data(symbol, start = balance[symbol][1], end = end)
    max_dd = current_price[symbol] / stop_loss_data.close.max() -1

    side = stop_loss_data.iloc[-120:]
    side['returns'] = side.close/ last_buy_price -1
    side['sideways'] = side.returns <0.1
    side['sideways_20'] = side.sideways.rolling(20).sum()
    
    # Stop Loss: sideways and dips
    if side.sideways_20.iloc[-1] == 20: 
        print(f'Sideways stop loss triggered for {symbol}')

    if max_dd < -0.1:
        print(f'Dip stop loss triggered for {symbol}')
        
    # Get data for other held symbols
    else:
        sell_return = current_price[symbol] / last_buy_price -1
        df = test_wrapper(symbol, start, end)
        sell_symbols.append(symbol)
        sell_returns.append(sell_return)
        sell_data = pd.concat([sell_data, df.iloc[-1,:-2].reset_index(drop=True)], axis = 1)
    
sell_data = sell_data.T
sell_data['sell_return'] = sell_returns
sell_data = sell_data.dropna()
sell_data.loc[len(sell_data)] = [0] * sell_data.shape[1] # add one row for terminal state

In [ ]:
# Initialise deployment environment and model
env = SellKnowledge(sell_data, train_mode = False)
model = PPO.load('sell_knowledge_agent')
actions = run_agent(env, model)

# Sell if P(sell) - P(hold) > 0.85
rank_sells = {}
for i in range(len(sell_symbols)):
    rank_sells[sell_symbols[i]] = actions[i]
rank_sells = pd.DataFrame(rank_sells).T.reset_index()
rank_sells['diff'] = rank_sells[0] - rank_sells[1]
rank_sells.to_csv('rank_sells.csv', index = False)
print(rank_sells)
rank_sells = rank_sells[rank_sells['diff'] > 0.85] 

In [ ]:
# Get data for buy signals
buy_data = pd.DataFrame()
buy_symbols = random.sample(buy_symbols, len(buy_symbols))
failed = set()

for symbol in buy_symbols:
    try:
        df = test_wrapper(symbol, start, end)
        buy_data = pd.concat([buy_data, df.iloc[-1,:-2].reset_index(drop=True)], axis = 1)
    except:
        print(f'Not enough data for {symbol}')
        failed.add(symbol)
        
buy_data = buy_data.T
buy_data.loc[len(buy_data)] = [0] * buy_data.shape[1] # add one row for terminal state 

# Initialise deployment environment and model
env = BuyKnowledge(buy_data, train_mode = False)
model = PPO.load('buy_knowledge_agent')
actions = run_agent(env, model)

# Prioritize buy signals with higher probability of high returns
rank_buys = {}
i = -1
for a in actions:
    i+=1
    if i in failed:
        continue
    else:
        rank_buys[buy_symbols[i]] = a
        
rank_buys = pd.DataFrame(rank_buys).T.reset_index()
rank_buys['diff'] = rank_buys[0] - rank_buys[1]
rank_buys = rank_buys.sort_values('diff', ascending = False).reset_index(drop = True) 
rank_buys.to_csv('rank_buys.csv', index = False)
print(rank_buys)
rank_buys = rank_buys[rank_buys['diff'] > 0] # increase threshold for lower risk

In [ ]:
rank_buys